In [258]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse
from transformers import AutoTokenizer
import time
from flash_attn import flash_attn_qkvpacked_func, flash_attn_func

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
# parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# args = parser.parse_args()

# Now we can use the argument value in our program.
# print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 4
block_size = 512
max_iters = 200
learning_rate = 2e-5
eval_iters = 100
n_embd = 256
n_head = 4
n_layer = 4
dropout = 0.2

print(device)

cuda


In [259]:
# Load the BERT uncased tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Get the vocabulary size
vocab_size = len(tokenizer)

In [260]:
# Encode and decode functions using the tokenizer
encode = lambda s: tokenizer.encode(s, add_special_tokens=True)
decode = lambda l: tokenizer.decode(l, skip_special_tokens=True)

In [261]:
def load_half_dataset_into_memory(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        f.seek(0, 2)  # Move the cursor to the end of the file
        half_point = f.tell() // 200  # Find the halfway point
        f.seek(0)  # Reset cursor to the beginning
        data = f.read(half_point)  # Read up to the halfway point
    
    return data


# Preprocess and encode your dataset (1/2 of it), then convert it to tensor
train_data = load_half_dataset_into_memory("output_train.txt")
val_data = load_half_dataset_into_memory("output_val.txt")

# Assuming you have a function encode() that converts text to a list of integers
train_encoded = torch.tensor(encode(train_data), dtype=torch.long)
val_encoded = torch.tensor(encode(val_data), dtype=torch.long)

Token indices sequence length is longer than the specified maximum sequence length for this model (406817 > 512). Running this sequence through the model will result in indexing errors


In [262]:
# def get_random_chunk(split):
#     filename = "train_split.txt" if split == 'train' else "val_split.txt"
#     with open(filename, 'rb') as f:
#         with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
#             file_size = len(mm)
#             start_pos = random.randint(0, file_size - block_size * batch_size)
            
#             # Ensure start position is at the beginning of a line to avoid partial sentences
#             if start_pos > 0:
#                 mm.seek(start_pos - 1)
#                 # Move to the next newline to avoid starting in the middle of a line
#                 while mm.read(1) != b"\n" and mm.tell() < file_size:
#                     pass
#                 start_pos = mm.tell()
            
#             end_pos = start_pos + block_size * batch_size
#             if end_pos > file_size:
#                 # Adjust start_pos if end_pos exceeds file size
#                 start_pos = max(0, file_size - block_size * batch_size)
#                 mm.seek(start_pos)
            
#             block = mm.read(block_size * batch_size)
#             decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
#             # Splitting into smaller chunks if necessary and encoding
#             data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
#     return data

# def get_batch(split):
#     valid_batch = False
#     while not valid_batch:
#         data = get_random_chunk(split)
#         if data.size(0) > block_size:
#             try:
#                 ix = torch.randint(0, data.size(0) - block_size, (batch_size,))
#                 x = torch.stack([data[i:i+block_size] for i in ix])
#                 y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#                 valid_batch = True
#             except RuntimeError as e:
#                 print(f"Error encountered: {e}. Trying to fetch a new chunk...")
#                 # This catch block will retry fetching a new chunk if the current one is not suitable
#         else:
#             print("Fetched chunk is too small, fetching a new chunk...")
#             # This else block will fetch a new chunk if the current one is too small
    
#     x, y = x.to(device), y.to(device)
#     return x, y

In [263]:
def get_batch(split):
    # start_time = time.time()
    # Select the appropriate dataset based on the split
    data = train_encoded if split == 'train' else val_encoded
    
    # Ensure we have enough data to sample from
    if data.size(0) > block_size:
        ix = torch.randint(0, data.size(0) - block_size, (batch_size,))
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    else:
        raise ValueError("Dataset size is too small for the requested block and batch sizes.")
    
    # Assuming 'device' is defined (e.g., 'cuda' or 'cpu')
    x, y = x.to(device), y.to(device)
    # print("--- %s seconds ---" % (time.time() - start_time))
    return x, y

In [264]:
class MHFlashAttn(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, n_head=n_head):
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(n_embd//n_head, head_size, bias=False, dtype=torch.float16)
        self.query = nn.Linear(n_embd//n_head, head_size, bias=False, dtype=torch.float16)
        self.value = nn.Linear(n_embd//n_head, head_size, bias=False, dtype=torch.float16)

        
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, n_head=n_head):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        x = x.view(B, T, n_head, C//n_head) # (B,T,C) -> (B,T,hs,C/hs)
        # print(x.shape)
        
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        # wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        # wei = F.softmax(wei, dim=-1) # (B, T, T)
        # wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        # out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        out = flash_attn_func(q, k, v, dropout_p=0.0, softmax_scale=None, causal=True,
                          window_size=(-1, -1), alibi_slopes=None, deterministic=False)
        return out
# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]


class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        # self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.mha_flash_attn = MHFlashAttn(head_size)
        # self.proj = nn.Linear(head_size * num_heads, n_embd, dtype=torch.float16)
        self.proj = nn.Linear(head_size, n_embd, dtype=torch.float16)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.mha_flash_attn(x)
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd, dtype=torch.float16),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd, dtype=torch.float16),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        # head_size = n_embd // n_head
        head_size = n_embd
        x_size = torch.tensor([batch_size, block_size, n_head, n_embd])
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        # self.ln1 = nn.LayerNorm(n_embd, dtype=torch.float16)
        # self.ln2 = nn.LayerNorm(n_embd, dtype=torch.float16)
        # self.ln1 = nn.LayerNorm(x_size[1:])
        # self.ln2 = nn.LayerNorm(x_size[1:])
        self.ln1 = nn.LayerNorm(n_embd, head_size, dtype=torch.float16)
        self.ln2 = nn.LayerNorm(n_embd, head_size, dtype=torch.float16)
    def forward(self, x):
        y = self.sa(x)
        print(y.shape)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, dtype=torch.float16)
        self.position_embedding_table = nn.Embedding(block_size, n_embd, dtype=torch.float16)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size, dtype=torch.float16)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)



In [265]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
    
            # Ensure we have enough data to sample from
            if val_encoded.size(0) > block_size:
                ix = torch.randint(0, val_encoded.size(0) - block_size, (batch_size,))
                x = torch.stack([val_encoded[i:i+block_size] for i in ix])
                y = torch.stack([val_encoded[i+1:i+block_size+1] for i in ix])
            else:
                raise ValueError("Dataset size is too small for the requested block and batch sizes.")
        
            logits, loss = model(x.to(device), y.to(device))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


    
    # Assuming 'device' is defined (e.g., 'cuda' or 'cpu')
    x, y = x.to(device), y.to(device)

In [266]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    # Ensure we have enough data to sample from
    if train_encoded.size(0) > block_size:
        ix = torch.randint(0, train_encoded.size(0) - block_size, (batch_size,))
        x = torch.stack([train_encoded[i:i+block_size] for i in ix])
        y = torch.stack([train_encoded[i+1:i+block_size+1] for i in ix])
    else:
        raise ValueError("Dataset size is too small for the requested block and batch sizes.")
    
    # evaluate the loss
    logits, loss = model.forward(x.to(device), y.to(device))
    # logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0
torch.Size([4, 512, 4, 256])


RuntimeError: The size of tensor a (512) must match the size of tensor b (4) at non-singleton dimension 2

In [ ]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)